# Processor Component

> UI component for displaying transcription in-progress state

In [ ]:
#| default_exp components.processor

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional
from fasthtml.common import *
from fasthtml.common import APIRouter

from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_styles
from cjm_fasthtml_daisyui.components.feedback.loading import loading, loading_styles, loading_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, h
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, text_align
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, flex_direction, items, justify, gap
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_workflow_transcription_single_file.core.html_ids import SingleFileHtmlIds
from cjm_fasthtml_workflow_transcription_single_file.core.config import SingleFileWorkflowConfig

## transcription_in_progress

Displays the transcription in-progress view with loading indicator, streaming text output, and cancel button.

In [ ]:
#| export
def transcription_in_progress(
    job_id: str,
    plugin_info: Dict[str, Any],
    file_info: Dict[str, Any],
    config: SingleFileWorkflowConfig,
    router: APIRouter,
):
    """Render transcription in-progress view with SSE updates.

    Args:
        job_id: Unique identifier for the transcription job.
        plugin_info: Dictionary with plugin details (id, title, supports_streaming).
        file_info: Dictionary with file details (name, path, type, size_str).
        config: Workflow configuration.
        router: Workflow router for generating route URLs.

    Returns:
        FastHTML component showing progress and SSE connection.
    """
    # Build URLs using router's .to() method for proper route generation
    sse_url = router.stream_job.to(job_id=job_id)
    cancel_url = router.cancel_job.to(job_id=job_id)

    return Div(
        Div(
            H2("Transcription in Progress", cls=str(card_title)),

            Div(
                # File and plugin info
                Div(
                    P(f"File: {file_info.get('name', 'Unknown')}",
                      cls=combine_classes(font_size.sm, m.b(1))),
                    P(f"Plugin: {plugin_info.get('title', 'Unknown')}",
                      cls=combine_classes(font_size.sm, m.b(4))),
                    cls=str(text_dui.base_content.opacity(80))
                ),

                # Status
                Div(
                    Span("Status: ", cls=str(font_weight.semibold)),
                    Span(
                        "Processing",
                        cls=combine_classes(badge, badge_colors.info),
                        id=SingleFileHtmlIds.PROCESSOR_STATUS_BADGE
                    ),
                    id=SingleFileHtmlIds.PROCESSOR_STATUS,
                    cls=str(m.b(4))
                ),

                # Loading indicator
                Div(
                    Span(
                        cls=combine_classes(
                            loading,
                            loading_styles.spinner,
                            loading_sizes.lg,
                            m.r(3)
                        )
                    ),
                    Div(
                        P("Transcribing audio...", cls=combine_classes(font_weight.semibold, m.b(1))),
                        P("This may take a moment depending on the file size.",
                          cls=combine_classes(font_size.sm, text_dui.base_content.opacity(60))),
                        cls=str(text_align.left)
                    ),
                    cls=combine_classes(flex_display, items.center, justify.center, m.b(6), p(6))
                ),

                # Cancel button
                Div(
                    Button(
                        "Cancel Transcription",
                        hx_post=cancel_url,
                        hx_target=SingleFileHtmlIds.as_selector(config.container_id),
                        hx_swap="innerHTML",
                        cls=combine_classes(btn, btn_colors.error, btn_sizes.sm),
                        id=SingleFileHtmlIds.CANCEL_BTN
                    ),
                    cls=str(text_align.center)
                ),

                cls=str(card_body)
            ),
            cls=combine_classes(card, bg_dui.base_200)
        ),

        # SSE connection for job completion updates
        Div(
            hx_ext="sse",
            sse_connect=sse_url,
            sse_swap="message",
            id=SingleFileHtmlIds.SSE_ELEMENT,
            style="display: none;"
        ),

        # Use workflow's container_id so HTMX outerHTML swap preserves the target
        id=config.container_id
    )

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()